In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

from keras.models import Sequential
model = Sequential()

from keras.layers import Dense, LSTM, Dropout

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range = (0,1))


In [ ]:
fake = pd.read_csv('../input/fake-and-real-news-dataset/Fake.csv')
true = pd.read_csv('../input/fake-and-real-news-dataset/True.csv')

In [ ]:
fake['true'] = 0
true['true'] = 1

df = pd.concat([fake, true], axis = 0)

In [ ]:
def create_num(x):
    text = []

    for i in range(len(x)):

        text.append(x[i].split())

    text = pd.DataFrame(text)

    text = text.stack().rank().unstack()
    text = text.fillna(-1)
    
    text = scaler.fit_transform(text)
    
    return text

In [ ]:

#text = create_num(df['text'].values)
#subject = create_num(df['subject'].values)


In [ ]:
text = pd.Series("Texan (36) suddenly forgets 20 years of his life")
inpt = pd.concat([df['title'], text])


x = create_num(inpt.values)
y = np.array(df['true'])

x_test = x[x.shape[0]-1]
x = np.delete(x, (x.shape[0]-1), axis = 0)


In [ ]:


model.add(Dense(500, input_shape = [1,42]))
model.add(Dropout(0.275))
model.add(Dense(250, activation = 'relu'))
model.add(Dropout(0.275))
model.add(Dense(125, activation = 'relu'))
model.add(Dropout(0.275))
model.add(Dense(1, activation = 'sigmoid'))

print(model.summary())

model.compile(loss = 'huber_loss',
             optimizer = 'adam',
             metrics = 'accuracy')

history = model.fit(
    x,
    y,
    batch_size=1024,
    epochs=100,
    verbose=1,
    validation_split=0.2,
    shuffle=True)


loss = history.history['loss']
print(plt.plot(loss))
model.save('./')

In [ ]:
x_test = x_test.reshape(1,-1)
prediction = model.predict(x_test)
result = np.around(prediction)
prediction = np.around(prediction,2) * 100

if result == 0:
    print("It's a fake by " + str(int(100-prediction[0][0])) + '% !')
else:
    print("It's true by " + str(int(prediction[0][0])) + '% !')
    
prediction

In [ ]:
x